In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [48]:
dataset = pd.read_csv('./CompleteDataset.csv')
X = dataset.iloc[:, :15].values
y = dataset.iloc[:, 15].values

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [50]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [51]:
classifier = Sequential()

In [52]:
classifier.add(Dense(units = 12, kernel_initializer = "uniform",  
                     activation = "sigmoid", input_dim = 15))

In [53]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

In [54]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

In [55]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [56]:
classifier.fit(X_train, y_train,  batch_size = 12, epochs =100)

Epoch 1/100
434/434 [==============================] - 1s 1ms/step - loss: 0.4886 - accuracy: 0.8145
Epoch 2/100
434/434 [==============================] - 1s 3ms/step - loss: 0.1488 - accuracy: 0.9562
Epoch 3/100
434/434 [==============================] - 2s 4ms/step - loss: 0.1183 - accuracy: 0.9606
Epoch 4/100
434/434 [==============================] - 1s 2ms/step - loss: 0.1110 - accuracy: 0.9627
Epoch 5/100
434/434 [==============================] - 2s 4ms/step - loss: 0.1071 - accuracy: 0.9637
Epoch 6/100
434/434 [==============================] - 1s 2ms/step - loss: 0.1042 - accuracy: 0.9650
Epoch 7/100
434/434 [==============================] - 2s 5ms/step - loss: 0.1029 - accuracy: 0.9664
Epoch 8/100
434/434 [==============================] - 1s 2ms/step - loss: 0.1012 - accuracy: 0.9667
Epoch 9/100
434/434 [==============================] - 1s 2ms/step - loss: 0.1002 - accuracy: 0.9691
Epoch 10/100
434/434 [==============================] - 2s 4ms/step - loss: 0.0990 - accura

In [60]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

41/41 [==============================] - 0s 1ms/step


In [61]:
scores = classifier.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

41/41 [==============================] - 0s 1ms/step - loss: 0.0743 - accuracy: 0.9808

accuracy: 98.08%


In [62]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[712,   4],
       [ 21, 564]])